In [62]:
import os
import pandas as pd
import datetime
import numpy as np
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display, HTML
import datetime
import matplotlib.pyplot  as plt
plt.style.use('ggplot')
# from utils import staff_code , SelectFilesButton

In [63]:
def load_resumen(path):
    if os.path.isfile(path) :
        resumen_df = pd.read_excel(path, sheet_name ='Resumen')
        resumen_df.columns = ['Fecha procesamiento','Dia','Cliente','Proyecto','Cantidad de horas','Valor hora','Total','Pagos / Anticipos','Saldo','Horas por mes']
        resumen_df = resumen_df[4:]
        resumen_df.drop(['Fecha procesamiento','Valor hora','Horas por mes', 'Total', 'Pagos / Anticipos', 'Saldo'], axis = 1, inplace= True)
        return resumen_df
    else:
        with output:
            print('Resumen no encontrado')
            print('Cargue el resumen primero')

In [64]:
def load_resumens():
    path = os.path.join('O:','Libovich','F','Estudio','Resumen')
    df = pd.DataFrame(columns = ['Dia', 'Cliente', 'Proyecto', 'Cantidad de horas', 'Arquitecto'])
    df.index.name = 'indice'
    for root, dirs, files in os.walk(path):
        for file in files:
            file_path = os.path.join(root,file)
            temp_df = load_resumen(file_path)
            temp_df['Arquitecto'] = str(file_path).split('-')[-1][:-5]
            if temp_df.shape[0] != 0: 
                df = pd.concat([df, temp_df])

    df['Dia'] =  pd.to_datetime(df['Dia'], format='%m/%d/%Y')
    df['Cantidad de horas'] = df['Cantidad de horas'].astype(float)
    df.reset_index(inplace = True)
    del df['index']
    return df

In [65]:
def plot_df_withax(df, color):
    fig, ax = plt.subplots(figsize=(9,6))
    df.plot(kind='barh',ax=ax, color=color)
    for p in ax.patches:
        ax.annotate(str(round(p.get_width(),2)), (p.get_width() * 1.005, p.get_y() * 1.05))
    ax.legend()
    #fig.

# Totales

In [71]:
class DataFPrint:
    def __init__(self):
        df_total = 0
        
        
    def set_total(self, df):
        self.df_total = df.copy()
        
    def set_proj(self, df , project):
        self.df_proj = df.copy()
        self.project = project
        
    def set_arq(self, df , arq):
        self.df_arq = df.copy()
        self.arq = arq
        
    def print_total(self):
        folder_path = os.path.join('O:','Libovich','F','Estudio','Resumen')
        self.df_total.to_excel(os.path.join(folder_path,'Analisis total.xlsx'))
        
    def print_proj(self):
        folder_path = os.path.join('O:','Libovich','F','Estudio','Resumen')        
        self.df_proj.to_excel(os.path.join(folder_path,'Analisis Proyecto {0!s}.xlsx'.format(self.project)))
        
    def print_arq(self):
        folder_path = os.path.join('O:','Libovich','F','Estudio','Resumen')
        self.df_arq.to_excel(os.path.join(folder_path,'Analisis arquitecto {0!s}.xlsx'.format(self.arq)))
        
def dowload_total(b):
    print_df.print_total()
    with output_total:
        display(HTML('Descargo el total en Analisis total.xlsx'))
    
def dowload_project(b):
    print_df.print_proj()
    with output_project : 
        display(HTML('Descargo en Analisis Proyecto {0!s}.xlsx'.format(print_df.project)))

def dowload_arq(b):
    print_df.print_arq()
    with output_arq : 
        display(HTML('Descargo en Analisis arquitecto {0!s}.xlsx'.format(print_df.arq)))

In [72]:
print_df = DataFPrint()
output_total = widgets.Output()
output_arq = widgets.Output()
output_project = widgets.Output()

In [73]:
def plot_filtered(begin_date,end_date, columna):
    df = load_resumens()
    df_filtered = df[(df['Dia']>pd.Timestamp(begin_date)) & (df['Dia']<pd.Timestamp(end_date))]
    plot_df_withax(df_filtered.groupby([columna]).sum()['Cantidad de horas'], 'blue')
    
    print_df.set_total(df_filtered.groupby([columna]).sum()['Cantidad de horas'])
    button = widgets.Button(description='Descargar')
    button.on_click(dowload_total)
    display(button)
        
    display(output_total)
    
_ = interact(plot_filtered,begin_date = widgets.DatePicker(value=pd.to_datetime(datetime.datetime.today().replace(month=datetime.datetime.today().month-1).strftime('%Y/%m/%d/')), description = 'Inicio'), 
             end_date= widgets.DatePicker(value=pd.to_datetime(datetime.datetime.today().strftime('%Y/%m/%d/')), description = 'Fin'),
            columna = ['Cliente','Proyecto','Arquitecto'])

interactive(children=(DatePicker(value=Timestamp('2020-01-10 00:00:00'), description='Inicio'), DatePicker(val…

## Proyectos por arquitecto

In [69]:
def plot_projects(begin_date,end_date, Arquitecto):
    df = load_resumens()
    df_filtered = df[(df['Dia']>pd.Timestamp(begin_date)) & (df['Dia']<pd.Timestamp(end_date))]
    df_filtered = df_filtered[(df_filtered['Arquitecto'] == Arquitecto)]
    
    print_df.set_arq(df_filtered.groupby(['Cliente','Proyecto']).sum()['Cantidad de horas'] , Arquitecto)
    button = widgets.Button(description='Descargar')
    button.on_click(dowload_arq)
    display(button)
    
    display(output_arq)
    
    total_horas = round(df_filtered['Cantidad de horas'].sum(),2)
    init_date = begin_date.strftime('%Y/%m/%d')
    last_date = end_date.strftime('%Y/%m/%d')
    print_string = str(f'<h3>Total de horas trabajadas por {Arquitecto} entre {init_date} y {last_date} son : {total_horas} hs.</h3>')
    display(HTML(print_string))
    plot_df_withax(df_filtered.groupby(['Cliente','Proyecto']).sum()['Cantidad de horas'], 'red')

_ = interact(plot_projects,begin_date = widgets.DatePicker(value=pd.to_datetime(datetime.datetime.today().replace(month=datetime.datetime.today().month-1).strftime('%Y/%m/%d/')), description = 'Inicio'), 
             end_date= widgets.DatePicker(value=pd.to_datetime(datetime.datetime.today().strftime('%Y/%m/%d/')), description = 'Fin'),
            Arquitecto = [i for i in load_resumens()['Arquitecto'].unique()])

interactive(children=(DatePicker(value=Timestamp('2020-01-10 00:00:00'), description='Inicio'), DatePicker(val…

## Arquitectos en cada proyecto

In [70]:
def plot_arquis(begin_date,end_date, Proyecto):
    df = load_resumens()
    df_filtered = df[(df['Dia']>pd.Timestamp(begin_date)) & (df['Dia']<pd.Timestamp(end_date))]
    df_filtered = df_filtered[(df_filtered['Proyecto'] == Proyecto)]
    
    print_df.set_proj(df_filtered.groupby(['Arquitecto']).sum()['Cantidad de horas'] , Proyecto)
    button = widgets.Button(description='Descargar')
    button.on_click(dowload_project)
    display(button)
    
    display(output_project)
    
    total_horas = round(df_filtered['Cantidad de horas'].sum(),2)
    init_date = begin_date.strftime('%d/%m/%Y')
    last_date = end_date.strftime('%d/%m/%Y')
    print_string = str(f'<h3>Total de horas trabajadas en {Proyecto} entre {init_date} y {last_date} son : {total_horas} hs.</h3>')
    display(HTML(print_string))
    plot_df_withax(df_filtered.groupby(['Arquitecto']).sum()['Cantidad de horas'], 'green')

_ = interact(plot_arquis,begin_date = widgets.DatePicker(value=pd.to_datetime(datetime.datetime.today().replace(month=1, day = 1).strftime('%Y/%m/%d/')), description = 'Inicio'), 
             end_date = widgets.DatePicker(value=pd.to_datetime(datetime.datetime.today().strftime('%Y/%m/%d/')), description = 'Fin'),
            Proyecto = [i for i in load_resumens()['Proyecto'].unique()])

interactive(children=(DatePicker(value=Timestamp('2020-01-01 00:00:00'), description='Inicio'), DatePicker(val…